# FastAI XLA Extensions Library

> The FastAI XLA Extensions library package allows your fastai/Pytorch models to run on TPUs using the Pytorch-XLA library.

## Install

`pip install git+https://github.com/butchland/fastai_xla_extensions`

## How to use

### Configure the Pytorch XLA package 

The Pytorch xla package requires an environment supporting TPUs (Kaggle kernels, GCP or Colab environments required)

If running on Colab, make sure the Runtime Type is set to TPU.


In [2]:
#hide
!pip install nbdev --upgrade > /dev/null

In [3]:
#hide
#colab
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
#hide
#colab
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [5]:
#colab
VERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4139  100  4139    0     0  21557      0 --:--:-- --:--:-- --:--:-- 21557
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200325 ...
     |████████████████████████████████| 61kB 2.6MB/s 
Uninstalling torch-1.5.1+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
Done updating TPU runtime
  Successfully uninstalled torch-1.5.1+cu101
Uninstalling torchvision-0.6.1+cu101:
  Successfully uninstalled torchvision-0.6.1+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly+20200325-cp36-cp36m-linux_x86_64.whl...
- [1 files][ 83.4 MiB/ 83.4 MiB]                                                
Operation completed over 1 objects/83.4 MiB.                             

Install fastai2 and the fastai_xla_extensions packages

In [19]:
!pip install fastai2 > /dev/null

In [20]:
!pip install git+https://github.com/butchland/fastai_xla_extensions > /dev/null

  Running command git clone -q https://github.com/butchland/fastai_xla_extensions /tmp/pip-req-build-0ytj2cpq


In [21]:
#hide
#colab
%cd /content/drive/My\ Drive/fastai_xla_extensions/

/content/drive/My Drive/fastai_xla_extensions


### Import the libraries
Import the pytorch xla, fastai2 and fastai_xla_extensions libraries

In [22]:
#colab
import torch_xla.core.xla_model as xm

In [23]:
from fastai2.vision.all import *

In [24]:
#colab
from fastai_xla_extensions.core import *

### Example
Build a Pets classifier -- adapted from fastai course [Lesson 5 notebook](https://github.com/fastai/course-v4/blob/master/nbs/05_pet_breeds.ipynb)

In [25]:
path = untar_data(URLs.PETS)/'images'

In [26]:
Path.BASE_PATH = path

In [27]:
pat = r'(.+)_\d+.jpg$'

In [28]:
datablock = DataBlock(
    blocks=(ImageBlock,CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(seed=42),
    get_y=using_attr(RegexLabeller(pat),'name'),
    item_tfms=Resize(460),
    batch_tfms=aug_transforms(size=224,min_scale=0.75)
)

Get a TPU device

In [29]:
#hide
tpu = 'cpu'
opt_func = Adam

In [30]:
#colab
tpu = xm.xla_device()

Set the dataloaders to use the TPU instead of a CPU

In [31]:
dls = datablock.dataloaders(path, device=tpu)

Wrap the optimizer function with the XLA Optimizer

In [32]:
#colab
opt_func = XLAOptFuncWrapper(Adam)

In [33]:
learner = cnn_learner(dls, resnet34, metrics=accuracy, opt_func=opt_func)
                      

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth


In [34]:
#colab
learner.fine_tune(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.865359,0.295098,0.907984,15:03


epoch,train_loss,valid_loss,accuracy,time
0,1.659670,2.236336,0.488498,14:13


## Status
The fastai XLA extensions library is still in very early development phase (not even alpha) which means there's still a lot of things not working. 

Use it at your own risk.

If you wish to contribute to the project, fork it and make pull request. 

This project uses [nbdev](https://nbdev.fast.ai/) -- a jupyter notebook first development environment and is being developed on [Colab](https://colab.research.google.com).


In [37]:
#hide
!nbdev_build_docs

converting: /content/drive/My Drive/fastai_xla_extensions/nbs/index.ipynb
converting /content/drive/My Drive/fastai_xla_extensions/nbs/index.ipynb to README.md
